## 1. Data preparation
Import packages

In [17]:
import folium
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import urllib.request  as urllib2
import pandas as pd
import json
import numpy as np
import requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import lxml
import requests

In [18]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

### 1.1. Read data about crime rate of cities

In [11]:
#Read the page
url_wiki = "https://en.wikipedia.org/wiki/List_of_United_States_cities_by_crime_rate"
critme_raw_df = pd.read_html(url_wiki, header=1)[0].copy()
critme_raw_df.head()

,State,City,Population,Total,Murder and Nonnegligent Manslaughter,Rape1,Robbery,Aggravated Assault,Total.1,Burglary,Larceny-Theft,Motor Vehicle Theft,Arson2
0,Alabama,Mobile3,248431,740.25,20.13,57.16,177.11,485.85,5453.83,1216.84,3730.21,506.78,22.94
1,Alaska,Anchorage,296188,1203.29,9.12,132.01,262.67,799.49,5415.82,748.17,3619.66,1047.98,20.93
2,Arizona,Chandler,249355,259.47,2.01,52.13,56.95,148.38,2329.61,314.41,1866.01,149.18,NaN
3,Arizona,Gilbert,242090,85.51,2.07,16.11,21.07,46.26,1385.85,192.49,1137.59,55.76,12.39
4,Arizona,Glendale,249273,488.22,4.81,38.91,192.96,251.53,4530.37,637.45,3426.36,466.56,19.26


In [12]:
print(critme_raw_df.count())

State                                   100
City                                    100
Population                              100
Total                                    96
Murder and Nonnegligent Manslaughter    100
Rape1                                    96
Robbery                                 100
Aggravated Assault                      100
Total.1                                 100
Burglary                                100
Larceny-Theft                           100
Motor Vehicle Theft                     100
Arson2                                   92
dtype: int64


#### Data cleaning
We will consider cities with a population of less than 500 thousand

In [26]:
critme_df = critme_raw_df
critme_df = critme_df[critme_df.Population < 500000]
critme_df = critme_df[['State', 'City', 'Population', 'Total', 'Total.1', 'Arson2']]
print(critme_df.count())
critme_df.head()

State         65
City          65
Population    65
Total         62
Total.1       65
Arson2        61
dtype: int64


,State,City,Population,Total,Total.1,Arson2
0,Alabama,Mobile3,248431,740.25,5453.83,22.94
1,Alaska,Anchorage,296188,1203.29,5415.82,20.93
2,Arizona,Chandler,249355,259.47,2329.61,NaN
3,Arizona,Gilbert,242090,85.51,1385.85,12.39
4,Arizona,Glendale,249273,488.22,4530.37,19.26


Drop NaN

In [27]:
critme_df = critme_df.dropna()
critme_df.head()

,State,City,Population,Total,Total.1,Arson2
0,Alabama,Mobile3,248431,740.25,5453.83,22.94
1,Alaska,Anchorage,296188,1203.29,5415.82,20.93
3,Arizona,Gilbert,242090,85.51,1385.85,12.39
4,Arizona,Glendale,249273,488.22,4530.37,19.26
5,Arizona,Mesa,492268,415.83,2171.99,4.67


Sum categories of crime

In [28]:
critme_df['Crime_Rate'] = critme_df['Total'] + critme_df['Total.1'] + critme_df['Arson2']
critme_df_result = critme_df[['State', 'City', 'Population', 'Crime_Rate']]
critme_df_result.head()
print(critme_df_result.count())

State         59
City          59
Population    59
Crime_Rate    59
dtype: int64


### 1.2 Create data abouf sport venues
#### Set geolocator, parametes to Foursquare API

In [24]:
geolocator = Nominatim(user_agent = 'test')
CLIENT_ID = '1ROEGGL32JNNXXQVZ1XAQM1RN11GXZVG0OPQYTXUES1NZWKJ'
CLIENT_SECRET = 'CV12M1JW2IX22BY2PDG1PPNPFBGRJIWIGOLYX5PFVXF4S1BE' 
VERSION = '20180604'
LIMIT = 1000

#### Count sport venues

In [20]:
def count_venues(lat, lng, category_id_list, radius=10000):
        
    counter_list = []
    
    for category_id in category_id_list:
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&categoryId={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT,
            category_id
        )
        
        count = 0
        # make the GET request
        try:
            result_get = requests.get(url).json()
            count = len(result_get["response"]['groups'][0]['items'])
        except:
            # retry one more time
            try:
                result_get = requests.get(url).json()
                count = len(result_get["response"]['groups'][0]['items'])
            except:
                print(result_get)
            
        counter_list.append(count)
    #print(counter_list)
    return counter_list

#### Define categories of sport venues using Foursquare list of venue categories:
https://developer.foursquare.com/docs/resources/categories

In [21]:
categories = {'athletics stadium': '4bf58dd8d48988d187941735',
              'badminton court': '52e81612bcbc57f1066b7a2b',
              'baseball field': '4bf58dd8d48988d1e8941735',
              'basketball court': '4bf58dd8d48988d1e1941735',
              'football field': '4cce455aebf7b749d5e191f5',
              'gym / fitness':  '4bf58dd8d48988d175941735',
              'hockey field': '56aa371be4b08b9a8d57352c',
              'rink': '4bf58dd8d48988d168941735',
              'ski tracking': '4eb1c0f63b7b52c0e1adc2eb',
              'sport club': '52e81612bcbc57f1066b7a2e',
              'swimming school': '52e81612bcbc57f1066b7a44',
              'tennis court': '4e39a956bd410d7aed40cbc3',
              'university gym': '4bf58dd8d48988d1b2941735',
              'volleyball court': '4eb1bf013b7b6f98df247e07'
             }

#### Get numbers of sport venues for each city

In [25]:
for state, c, p, cr in zip(critme_df_result['State'], critme_df_result['City'], critme_df_result['Population'], critme_df_result['Crime_Rate']):
    city = ''.join([i for i in c if not i.isdigit()])
    location = geolocator.geocode(state + ' ' + city)
    if (location):
        print((city, location.latitude, location.longitude))
        count_venues(location.latitude, location.longitude, categories.values())
    else:
        print(city + ' cannot get location')

('Mobile', 30.6943566, -88.0430541)
{'meta': {'code': 429, 'requestId': '5c7a3265db04f505a318163d', 'errorDetail': 'Quota exceeded', 'errorType': 'quota_exceeded'}, 'response': {}}
{'meta': {'code': 429, 'requestId': '5c7a3266351e3d13ab352d20', 'errorDetail': 'Quota exceeded', 'errorType': 'quota_exceeded'}, 'response': {}}
{'meta': {'code': 429, 'requestId': '5c7a3267351e3d13a5394e8b', 'errorDetail': 'Quota exceeded', 'errorType': 'quota_exceeded'}, 'response': {}}
{'meta': {'code': 429, 'requestId': '5c7a32684c1f6762f712ae48', 'errorDetail': 'Quota exceeded', 'errorType': 'quota_exceeded'}, 'response': {}}
{'meta': {'code': 429, 'requestId': '5c7a32691ed2196e461e3880', 'errorDetail': 'Quota exceeded', 'errorType': 'quota_exceeded'}, 'response': {}}
{'meta': {'code': 429, 'requestId': '5c7a326a6a60714c84d8f579', 'errorDetail': 'Quota exceeded', 'errorType': 'quota_exceeded'}, 'response': {}}
{'meta': {'code': 429, 'requestId': '5c7a326c4434b9578554be94', 'errorDetail': 'Quota exceeded

KeyboardInterrupt: 

In [71]:
a = [['a', 100, 1, 2, 3, 4, 5], ['b', 101, 10, 20, 30, 40, 50]]

df = pd.DataFrame(a, columns=['N', 'P', 'A', 'B', 'C', 'D', 'E'])

In [72]:
df.head()

,N,P,A,B,C,D,E
0,a,100,1,2,3,4,5
1,b,101,10,20,30,40,50


In [73]:
df.iloc[:,2:] = df.iloc[:,2:].div(df.P, axis=0)

In [74]:
df.head()

,N,P,A,B,C,D,E
0,a,100,0.01000,0.02000,0.03000,0.04000,0.05000
1,b,101,0.09901,0.19802,0.29703,0.39604,0.49505


In [83]:
df.loc[:,['A', 'B']]

,A,B
0,0.01000,0.02000
1,0.09901,0.19802
